# COVID-19 Daily Report All Regions
### Deployment on web server (local host or cloud provider)

## Author: Prasanna Badami

### <font size = '4'> Note:
<font size = '3'> 1. This source code runs in Jupyter notebook with environment meeting all requirements libraries used in this source code. <br>
<font size = '3'> 2. This file needs internet connection, it downloads an input file from CSSEGISandData repository on GitHub. Internet charges may apply. <br>
    ###################### Important ########################################## <br>
<font size = '3'> 3. Standalone HTML will not get created, because the code is in pure Python. <br>
<font size = '3'> 4. To get the output in web browser similar to standalone interactive HTML without exposing code, you need to use Bokeh server deployment, which deploys output of this source code on local system or cloud provider. This is done by adding panel.servable() in source file & launching the app using 'panel serve sourcefile.ipynb' at command prompt 

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [ ]:
from bokeh.models import HoverTool, NumeralTickFormatter, Div
from bokeh.layouts import gridplot
import holoviews as hv
hv.extension('bokeh')
import panel as pn
pn.extension()
import hvplot.pandas

In [ ]:
date_on_file_today = datetime.today().strftime('%m-%d-%Y')

In [ ]:
date_on_file_yesterday =  (datetime.today() - timedelta(days = 1)).strftime('%m-%d-%Y')

In [ ]:
date_on_file_day_b4_yesterday = (datetime.today() - timedelta(days = 2)).strftime('%m-%d-%Y')

In [ ]:
# First try to get file with today's date, if not available,
# try with yesterday's date, if not available,
# try with day before yesterday's date, if not available 
# assign empty dataframe to df_daily_report, this dataframe is for checked contents in next cell.

In [ ]:
date_on_file_yesterday =  (datetime.today() - timedelta(days = 1)).strftime('%m-%d-%Y')

In [ ]:
date_onfile_day_b4_yesterday = (datetime.today() - timedelta(days = 2)).strftime('%m-%d-%Y')

In [ ]:
date_on_report = 'today'

In [ ]:
try:
    df_daily_report = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'+ date_on_file_today + '.csv')
    date_on_report = date_on_file_today
    print('Selected %s.csv' % date_on_file_today)
except:
    try:
        df_daily_report = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'+ date_on_file_yesterday + '.csv')
        date_on_report = date_on_file_yesterday
        print('Selected %s.csv' % date_on_file_yesterday)
    except:
        try:
            df_daily_report = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'+ date_on_file_day_b4_yesterday + '.csv')
            date_on_report = date_on_file_day_b4_yesterday
            print('Selected %s.csv' % date_on_file_day_b4_yesterday)
        except:
            df_daily_report = pd.DataFrame()
            print('HTTPError: HTTP Error 404: File Not Found')

In [ ]:
# df_daily_report

In [ ]:
pn_dashboard = pn.Column(Div(text = 'HTTPError: HTTP Error 404: Input file for ' + str(date_on_file_today) + ' or ' \
                             +  str(date_on_file_yesterday) + ' or ' + str(date_on_file_day_b4_yesterday) + ' not found on GitHub', width = 925))
if (df_daily_report.empty == True):
#     Disable GUI & all analysis here as file is not found.
    print('Since file is not found on server, dataframe is assigned to empty. Not proceeding with further analysis')
else:
    last_update = df_daily_report['Last_Update'][0]
    print('Last update date: ' + last_update)
    metrics= ['Active', 'Deaths', 'Recovered', 'Confirmed']
    # make sure variable names are proper & uniform..
    country_name = pn.widgets.Select(name = 'Country', options = df_daily_report['Country_Region'].unique().tolist(), width = 250)

    # Update province_state dynamically based on country_region later in the code
    province_state = pn.widgets.Select(name = 'Province', options = [' '], width = 275)
    # Update Admin2 based on province_stat2
    admin2 = pn.widgets.Select(name = 'County', options = [' '], width = 200)
    
    footer = Div(text = """Graphic: Prasanna Badami 
                    <div>Source: Center for Systems Science & Engineering, Johns Hopkins University</div> 
                    <div>Last update: {} </div> 
                    <div style = "font-size:12px"> Note:</div>
                    <div style = "font-size:12px"><i><t>Incident rate is calculated as confirmed cases per 100,000 population.</i></div>
                    <div style = "font-size:12px"><i> Case fatality ratio is the ratio of deaths to confirmed cases.</i></div>
                    """.format(last_update),  
                 width = 1230,
                 background = 'white',
                )

    hover_tool_daily = HoverTool(tooltips = [('Metric', '@Variable'), ('Cases', '@value{0,}')])

    @pn.depends(country_name.param.value)
    def UpdateProvinceList(country_name):
    #     First select country
        select_country = df_daily_report[df_daily_report['Country_Region'].isin([country_name])]
    #     Select provinces of the country
        checkprovince4nan = select_country['Province_State'].unique().tolist()
        province_state.options = checkprovince4nan

    @pn.depends(province_state.param.value)
    def UpdateAdmin2List(province_name):
    #     First select country
        select_country = df_daily_report[df_daily_report['Country_Region'].isin([country_name.value])]
    #     Select counties of the province of the country
        checkcounties4nan = select_country[select_country['Province_State'].isin([province_name])]['Admin2'].values.tolist()
        admin2.options = checkcounties4nan


    @pn.depends(province_state.param.value, country_name.param.value, admin2.param.value)
    def UpdatePlot(province_state, country_name, admin2):
    #     Refer to dataframe to understand below comments.
    #     Select the country, province & county to display
    #     You will displaying which is not 'nan' in order country, province, county with column 'metrics', selected in hvplot.
    #     if province is 'nan' & admin2 is 'nan', you will anyway displaying the graph at the country level.
    #     if province is not 'nan' & admin2 is 'nan', you will be displaying the graph at the province level
    #     if province is not 'nan' & admin2 is not 'nan', you will be displaying the graph at the admin2 level
    #     No need for checking for using conditional statements to check for 'nan', it doesn't matter
        select_country = df_daily_report[df_daily_report['Country_Region'].isin([country_name])]
        select_province = select_country[select_country['Province_State'].isin([province_state])]
        plot = select_province[select_province['Admin2'].isin([admin2])].hvplot(kind = 'bar', x = 'Country_Region', y = metrics,
                                                                                ylabel = 'COVID-19 Cases', xlabel = 'Country_Region',
                                                                                color = ['orange', 'red', 'lawngreen','royalblue']
                                                                               )

        bkf = hv.render(plot.opts(width = 800, height = 500, bar_width = 0.4, tools = [hover_tool_daily]))
       
        bkf.toolbar_location = None
        
        bkf.xaxis.axis_label_text_font_style = 'bold'
        bkf.xaxis.major_label_text_font_style = 'bold'
    
        bkf.y_range.start = 0
        bkf.yaxis.formatter = NumeralTickFormatter(format = 'a')
        bkf.yaxis.axis_label_text_font_style = 'bold'
        bkf.yaxis.major_label_text_font_style = 'bold'
        
        
        bkt = hv.render(hv.Table(select_province[select_province['Admin2'].isin([admin2])][['Incident_Rate', 'Case_Fatality_Ratio']].round(2)))
        if (str(province_state) == 'nan'):
            bkf.title.text = 'Daily Report: ' + country_name + ' | ' +  'Last update: ' + str(last_update)
        elif(str(admin2) == 'nan'):
            bkf.title.text = 'Daily Report: ' + country_name + ' | ' + 'Province: ' + str(province_state) + ' | ' + 'Last update: ' + str(last_update)
        else:
            bkf.title.text = 'Daily Report: ' + country_name + ' | ' + 'Province: ' + str(province_state) + ' | ' + 'County: ' + str(admin2) + ' | ' + 'Last update: ' + str(last_update)

        return(gridplot(children = [[bkf, bkt]], merge_tools = False))

    pn_dashboard = pn.Column(pn.WidgetBox(pn.Row(country_name, province_state, admin2), UpdateProvinceList, UpdateAdmin2List, UpdatePlot, footer))

# To deploy this source code on local system or cloud using Bokeh server
pn_dashboard.servable(title = 'COVID-19_Daily_Report_All_Regions_'+str(date_on_report))


# To launch the server, run command 'panel serve COVID-19_Daily_Report_All_Regions_v2.ipynb' at command prompt
# The app is deployed (default port: 5006) at http://localhost:5006/COVID-19_Daily_Report_All_Regions_v2

In [ ]:
pn_dashboard